In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/test_y')
train_x, train_y, test_x, test_y

(dask.array<from-npy-stack, shape=(2400, 128, 128, 1), dtype=float32, chunksize=(2400, 128, 128, 1)>,
 dask.array<from-npy-stack, shape=(2400, 128, 128, 1), dtype=float32, chunksize=(2400, 128, 128, 1)>,
 dask.array<from-npy-stack, shape=(1600, 128, 128, 1), dtype=float32, chunksize=(1600, 128, 128, 1)>,
 dask.array<from-npy-stack, shape=(1600, 128, 128, 1), dtype=float32, chunksize=(1600, 128, 128, 1)>)

In [3]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(n_filters=16, dropout=0.5):
    image_input = tf.keras.Input(shape=(128, 128, 1), name='input_layer')
    
    conv_1 = build_block(image_input, n_filters)
    conv_2 = build_block(conv_1, n_filters)
    pool_1 = tf.keras.layers.MaxPooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)
    conv_4 = build_block(conv_3, n_filters * 2)
    pool_2 = tf.keras.layers.MaxPooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)
    conv_6 = build_block(conv_5, n_filters * 4)
    pool_3 = tf.keras.layers.MaxPooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)
    conv_8 = build_block(conv_7, n_filters * 8)
    pool_4 = tf.keras.layers.MaxPooling2D(padding='same')(conv_8)
    drop_4 = tf.keras.layers.SpatialDropout2D(dropout)(pool_4)
    
    conv_9 = build_block(drop_4, n_filters * 16)
    conv_10 = build_block(conv_9, n_filters * 16)
    
    donv_1 = tf.keras.layers.Convolution2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(conv_10)
    donv_1 = tf.keras.layers.concatenate([donv_1, conv_8])
    donv_1 = tf.keras.layers.SpatialDropout2D(dropout)(donv_1)
    conv_11 = build_block(donv_1, n_filters * 8)
    conv_12 = build_block(conv_11, n_filters * 8)
    
    donv_2 = tf.keras.layers.Convolution2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(conv_12)
    donv_2 = tf.keras.layers.concatenate([donv_2, conv_6])
    donv_2 = tf.keras.layers.SpatialDropout2D(dropout)(donv_2)
    conv_13 = build_block(donv_2, n_filters * 4)
    conv_14 = build_block(conv_13, n_filters * 4)
    
    donv_3 = tf.keras.layers.Convolution2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(conv_14)
    donv_3 = tf.keras.layers.concatenate([donv_3, conv_4])
    donv_3 = tf.keras.layers.SpatialDropout2D(dropout)(donv_3)
    conv_15 = build_block(donv_3, n_filters * 2)
    conv_16 = build_block(conv_15, n_filters * 2)
    
    donv_4 = tf.keras.layers.Convolution2DTranspose(n_filters, (3, 3), strides = (2, 2), padding = 'same')(conv_16)
    donv_4 = tf.keras.layers.concatenate([donv_4, conv_2])
    donv_4 = tf.keras.layers.SpatialDropout2D(dropout)(donv_4)
    conv_17 = build_block(donv_4, n_filters)
    conv_18 = build_block(conv_17, n_filters)
    
    output = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer='glorot_normal', activation='sigmoid')(conv_18)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model

In [4]:
model = build_unet()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 16) 144         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 16) 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [5]:
epochs, batch_size, lr, filepath = 10000, 16, 0.001, "day_1.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True)
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = zip(image_generator, mask_generator)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=batch_size)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = tf.keras.callbacks.TensorBozard(log_dir=os.getcwd())

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    #initial_epoch=65,
                    callbacks=[checkpoint, tb])

Epoch 1/10000
149/150 [============================>.] - ETA: 0s - loss: 9.1026 - acc: 0.6957
Epoch 00001: val_acc improved from -inf to 0.23291, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 59s 396ms/step - loss: 9.0863 - acc: 0.6969 - val_loss: 36.0130 - val_acc: 0.2329
Epoch 2/10000
149/150 [============================>.] - ETA: 0s - loss: 7.5180 - acc: 0.7732
Epoch 00002: val_acc improved from 0.23291 to 0.27694, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 27s 180ms/step - loss: 7.5313 - acc: 0.7729 - val_loss: 21.4759 - val_acc: 0.2769
Epoch 3/10000
149/150 [============================>.] - ETA: 0s - loss: 7.0119 - acc: 0.7869
Epoch 00003: val_acc improved from 0.27694 to 0.78389, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 27s 181ms/step - loss: 7.0119 - acc: 0.7869 - val_loss: 6.6902 - val_acc: 0.7839
Epoch 4/10000
149/150 [============================>.] - ETA: 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



149/150 [============================>.] - ETA: 0s - loss: 2.8244 - acc: 0.9028
Epoch 00605: val_acc did not improve from 0.94167
150/150 [==============================] - 30s 198ms/step - loss: 2.8227 - acc: 0.9028 - val_loss: 2.1150 - val_acc: 0.9402
Epoch 606/10000
149/150 [============================>.] - ETA: 0s - loss: 2.7401 - acc: 0.9037
Epoch 00606: val_acc did not improve from 0.94167
150/150 [==============================] - 30s 198ms/step - loss: 2.7349 - acc: 0.9038 - val_loss: 2.5497 - val_acc: 0.9330
Epoch 607/10000
149/150 [============================>.] - ETA: 0s - loss: 2.7623 - acc: 0.9042
Epoch 00607: val_acc did not improve from 0.94167
150/150 [==============================] - 30s 198ms/step - loss: 2.7847 - acc: 0.9036 - val_loss: 2.1487 - val_acc: 0.9392
Epoch 608/10000
149/150 [============================>.] - ETA: 0s - loss: 2.7700 - acc: 0.9035
Epoch 00608: val_acc did not improve from 0.94167
150/150 [==============================] - 30s 198ms/step - 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



149/150 [============================>.] - ETA: 0s - loss: 2.6699 - acc: 0.9073
Epoch 00667: val_acc did not improve from 0.94167
150/150 [==============================] - 30s 198ms/step - loss: 2.6732 - acc: 0.9072 - val_loss: 2.3783 - val_acc: 0.9354
Epoch 668/10000
149/150 [============================>.] - ETA: 0s - loss: 2.6980 - acc: 0.9049
Epoch 00668: val_acc did not improve from 0.94167
150/150 [==============================] - 30s 198ms/step - loss: 2.6948 - acc: 0.9050 - val_loss: 2.2199 - val_acc: 0.9385
Epoch 669/10000
149/150 [============================>.] - ETA: 0s - loss: 2.6880 - acc: 0.9058
Epoch 00669: val_acc did not improve from 0.94167
150/150 [==============================] - 30s 198ms/step - loss: 2.6808 - acc: 0.9061 - val_loss: 2.4988 - val_acc: 0.9340
Epoch 670/10000
149/150 [============================>.] - ETA: 0s - loss: 2.6545 - acc: 0.9091
Epoch 00670: val_acc did not improve from 0.94167
150/150 [==============================] - 30s 198ms/step - 

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



149/150 [============================>.] - ETA: 0s - loss: 2.4529 - acc: 0.9129
Epoch 00956: val_acc did not improve from 0.94567
150/150 [==============================] - 29s 194ms/step - loss: 2.4617 - acc: 0.9127 - val_loss: 1.9951 - val_acc: 0.9426
Epoch 957/10000
149/150 [============================>.] - ETA: 0s - loss: 2.4822 - acc: 0.9117
Epoch 00957: val_acc did not improve from 0.94567
150/150 [==============================] - 29s 191ms/step - loss: 2.4909 - acc: 0.9114 - val_loss: 2.2081 - val_acc: 0.9390
Epoch 958/10000
149/150 [============================>.] - ETA: 0s - loss: 2.3652 - acc: 0.9138
Epoch 00958: val_acc did not improve from 0.94567
150/150 [==============================] - 29s 191ms/step - loss: 2.3787 - acc: 0.9136 - val_loss: 2.0291 - val_acc: 0.9425
Epoch 959/10000
149/150 [============================>.] - ETA: 0s - loss: 2.4349 - acc: 0.9121
Epoch 00959: val_acc did not improve from 0.94567
150/150 [==============================] - 29s 191ms/step - 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_generator.py", line 171, in fit_generator
    x, y, sample_weight=sample_weight, class_weight=class_weight)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1828, in train_on_batch
    outputs = self.train_function(ins)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/backend.py", line 2978, in __call__
    run_metadata=self.run_metadata)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1399, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/s

TypeError: must be str, not list